In [1]:
import numpy as np
import serial
import datetime
from time import sleep
import pandas as pd
from pathlib import Path

In [8]:
sr542 = serial.Serial(port='COM4',baudrate=115200,timeout=20)
print(sr542.name)

COM4


In [3]:
def serialQry(port, command):
    port.reset_input_buffer()
    port.write((command + '\r').encode('ascii'))
    return port.readline().decode('ascii').strip() #strip removes white space

def serialCmd(port, command):
    port.reset_input_buffer()
    port.write((command + '\r').encode('ascii'))
    
def serialRead(port):
    return port.readline().decode('ascii').strip()

In [9]:
serialQry(sr542, '*IDN?')

'Stanford_Research_Systems,SR542,s/n00000001,v0.10'

In [10]:
serialCmd(sr542, '*CLS')

## Set up the instrument

In [11]:
# enable factory mode
serialCmd(sr542, 'BSRS 1, 314159265')
serialQry(sr542, 'BSRS?')

'1'

In [10]:
isoDate = datetime.date.today().isocalendar()
weekNum = f'{isoDate[1]:02}'
yearLast2 = str(isoDate[0])[2:]

n = 1 # TODO: get the count by incrementing from a database?
serialCountStr = f'{n:04}'

# 1. Construct the date code
chopperSerialNum = yearLast2 + weekNum + serialCountStr
print(chopperSerialNum)

21010001


In [12]:
# 0. Check if Chopper Head already has uploaded CAL data
serialCmd(sr542, '$CHT DNLD')

In [13]:
serialQry(sr542, 'LERR?')

'0'

In [14]:
sn = serialQry(sr542, '$CSR?')
sn

'20460001'

In [87]:
#OPTIONAL: Erase All Chopper Head Data
serialCmd(sr542, '$CHE')

In [98]:
# 1. Write the chopper head serial number to RAM
serialCmd(sr542, f'$CSR {chopperSerialNum}')

In [99]:
serialQry(sr542, '$CSR?')

'20490001'

In [15]:
# Check the shaft index stored in RAM (should be zero at power-on)
serialQry(sr542, '$CIR?')

'65'

In [94]:
# 2. Perform a shaft calibration
serialCmd(sr542, '$SHC')
serialRead(sr542)
serialCmd(sr542, 'STOP')

In [60]:
# or manually write it (for testing)
serialCmd(sr542, '$CIR 0')

In [61]:
# Double check the shaft index
serialQry(sr542, '$CIR?')

'0'

In [15]:
# Read in data (for now from the .c files I used to use as part of the firmware source code)
file_dir = Path('./CalData')
filename = f'tickPos_{sn}.csv'
df = pd.read_csv((file_dir / filename), sep=',', skiprows=0, index_col=False, names = ['tick pos (revs)'], nrows=400)
df

,tick pos (revs)
0,0.000000
1,0.002519
2,0.005145
3,0.007505
4,0.009984
...,...
395,0.987530
396,0.990005
397,0.992532
398,0.995151


In [16]:
# 3. Write tick position cal data to RAM
for i, calWord in enumerate(df['tick pos (revs)']):
    serialCmd(sr542, f'$TIK {i}, {calWord:e}')
    sleep(.05)

In [16]:
serialQry(sr542, '$TIK? 2')

'5.145447e-03'

In [18]:
file_dir = Path('./CalData')
filename = f'currCorr_{sn}.csv'
df = pd.read_csv((file_dir / filename), sep=',', skiprows=0, index_col=False, names = ['current corr (frac)'], nrows=400)
df

,current corr (frac)
0,0.001798
1,0.001780
2,0.001812
3,0.001750
4,0.001756
...,...
395,0.001798
396,0.001813
397,0.001822
398,0.001857


In [19]:
#4. Write current compensation cal data to RAM
for i, calWord in enumerate(df['current corr (frac)']):
    serialCmd(sr542, f'$CCR {i}, {calWord:e}')
    sleep(.05)

In [20]:
serialQry(sr542, '$CCR? 2')

'1.811830e-03'

### Write all the data to the chopper head

In [21]:
serialCmd(sr542, '$CHT UPLD')

### Restart the controller

(May need to re-establish communication at that point too)

In [22]:
sr542 = serial.Serial(port='COM4',baudrate=115200,timeout=20)
print(sr542.name)

COM4


In [23]:
serialCmd(sr542, '$CHT DNLD')

In [24]:
# Check all the chopper head cal values in RAM
serialQry(sr542, '$CSR?')

'20460001'

In [25]:
serialQry(sr542, '$CIR?')

'65'

In [26]:
serialQry(sr542, '$CCR? 399')

'1.776127e-03'

In [27]:
serialQry(sr542, '$TIK? 1')

'2.519182e-03'

In [271]:
serialCmd(sr542, '*CLS')

In [113]:
# Download all data from Chopper Head
serialCmd(sr542, '$CHT DNLD')

In [114]:
serialQry(sr542, 'LERR?')

'0'

In [17]:
serialQry(sr542, '$tcr?')

'0.900000, 10'

In [31]:
serialCmd(sr542, '$tcr 0.85, 0')